# Backup code

In [ ]:
# BACKUP CODE
def trainOneEpoch_OL(model, x_train, x_test, y_train, y_test, features_saved, labels_features_saved, batch_size):

    # RINOMINATO FEATURES -> SAVED FEATURES
       
    cntr = 1 #  A COSA SERVE?
    learn_rate = model.l_rate
    
    train_samples = x_train.shape[0]
    test_samples = x_test.shape[0]
    tot_samples = train_samples + test_samples
    
    # Merge train and test arrays
    x_tot = np.concatenate((x_train, x_test))   # Images
    y_tot = np.concatenate((y_train, y_test))   # Labels
    
    n_cluster = 10
    err_tot = 0
    cntr_clus = 0
    reminder = tot_samples%batch_size
    max_iter = int(tot_samples//batch_size)

    features_images = np.zeros(tot_samples)
        
    # FEATURE EXTRACTION
    # Estrae le features per ciascuna immagine utilizzando il Frozen Model
    print('**********************************\n Performing features extraction \n')

    for i in range(0, tot_samples):
        features = model.ML_frozen.predict(x_tot[i].reshape((1,28,28,1)), verbose = False)

        if i == 0:
          features_images = np.copy(features)       
        else:
          features_images = np.concatenate((features_images, features))

    # CLUSTERING
    print('**********************************\n Performing clustering\n')

    # Pseudo-labels
    for i in range(0, max_iter):
      
        # pseudo_label, err = k_mean_clustering(features_images[i*batch_size:i*batch_size+batch_size], features_saved, y_tot[i*batch_size:i*batch_size+batch_size], labels_features, n_cluster, batch_size)
        pseudo_label, err = k_mean_clustering(features_images[i*batch_size:i*batch_size+batch_size], features_saved, y_tot[i*batch_size:i*batch_size+batch_size], labels_features_saved, n_cluster, batch_size)
       
        err_tot += err
        pseudo_label = pseudo_label.astype(int)

        if i == 0:
          pseudo_labels = np.copy(pseudo_label)      
        else:
          pseudo_labels = np.append(pseudo_labels, pseudo_label)

        #print(f"\r    Currently at {np.round(np.round(cntr_clus/tot_samples*batch_size,4)*100,2)}% of dataset", end="")
        #print("\n")
        cntr_clus +=1

        # k_mean_clustering(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size):
    
        print("Current batch errors:", str(err) + "/"+ str(batch_size), "(accuracy " + str((1 - err/batch_size)*100) +")%")


    # Pseudo-labels for last batch
    
    if reminder != 0: 
        pseudo_label, err = k_mean_clustering(features_images[max_iter*batch_size:tot_samples], features_saved, y_tot[max_iter*batch_size:tot_samples], labels_features_saved, n_cluster, reminder)
        err_tot = err_tot + err
        pseudo_labels = np.append(pseudo_labels, pseudo_label)

        print("Current batch errors:", err)

    # Check pseudo-labels and errors in clustering
    
    #print("Pseudo_labels vector: ")
    #print(pseudo_labels)
    #print("\n")

    print("The error in clustering is: ")
    print(int(err_tot/tot_samples*100), '%')
    print("errors:", err_tot)
    print("tot samples", tot_samples)

    # ONLINE-LEARNING

    print('**********************************\n Performing training with OL\n')

    for i in range(0, tot_samples):

        CheckLabelKnown(model, pseudo_labels[i])
    
        y_true_soft = DigitToSoftmax(pseudo_labels[i], model.label)
               
        # Prediction
        
        y_pred = model.predict(features_images[i,:])
        
        # Backpropagation
        cost = y_pred-y_true_soft
        
        for j in range(0,model.W.shape[0]):

            # Update weights
            dW = np.multiply(cost, features_images[i,j]*learn_rate)
            model.W[j,:] = model.W[j,:]-dW

        # Update biases
        db      = np.multiply(cost, learn_rate)
        model.b = model.b-db
        
        # the next part is only to plot the confusion matrix
        # if the train data is finished still train the model but do not save the results
        if(i>=train_samples):

            y_true_soft = DigitToSoftmax(y_tot[i], model.label)
                   
            # Find the max iter for both true label and prediction
            if(np.amax(y_true_soft) != 0):
                max_i_true = np.argmax(y_true_soft)

            if(np.amax(y_pred) != 0):
                max_i_pred = np.argmax(y_pred)

            # Fill up the confusion matrix
            #for k in range(0,len(model.label)):
            #    if(model.label[max_i_pred] == model.std_label[k]):
            #        p = np.copy(k)
            #    if(model.label[max_i_true] == model.std_label[k]):
            #        t = np.copy(k)
            #
            #model.conf_matr[t,p] += 1  

        # ComputeEvalMetrics(y_pred)

        #print(f"\r    Currently at {np.round(np.round(cntr/tot_samples,4)*100,2)}% of dataset", end="")
        #print("\n")
        #cntr +=1

    '''Function to compute kmean clustering on the new dataset and the saved features'''
def k_mean_clustering(features_run, features_saved, labels_features_run, labels_features_saved, n_cluster, batch_size):

  # Define initial set of features
  labels_init_list = list(range(0, n_cluster))

  # labels_init_list = list([1, 9, 5, 0])
  # labels_init_list = list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
  # n_cluster = len(labels_init_list)

  # Extract from the saved features the labels that we need
  features_saved_init = []
  labels_features_saved_init = []
  # Extract features of digits considered in labels_init_list
  for i in range(0, len(features_saved)):
      if labels_features_saved[i] in labels_init_list:
        features_saved_init.append(features_saved[i,:])
        labels_features_saved_init.append(labels_features_saved[i])
  
  # Convert list to nparray
  features = np.array(features_saved_init)
  features = features.astype('float32')
  labels_features = np.array(labels_features_saved_init)  

  # Concateno al vettore delle features iniziali le features della nuova batch da analizzare
  features = np.concatenate((features, features_run))
  labels_features = np.append(labels_features, labels_features_run).astype(int)

  # Repeat until clustering is correct
  while True:
    # KMean Clustering
    k_mean = create_k_mean(features, n_cluster)

    # Find pseudolabels for each new image
    # Pseudolabels are computed by looking at the confusion matrix of the saved dataset (where ground truth is known)
    clusters_features_saved = list(k_mean.labels_[0:len(labels_features_saved_init)])
    labels_features_saved_init = list(labels_features_saved_init)
    cluster_list = list(range(0,n_cluster))
    map_clu2lbl, map_lbl2clu = cluster_to_label(clusters_features_saved, labels_features_saved_init, cluster_list, labels_init_list) 
 
    if len(map_clu2lbl) == n_cluster:
        # Exit the loop
        break

  clusters_features = k_mean.labels_

  # Find pseudo label (labels obtained from the model of each image
  pseudolabels_features = np.zeros(len(clusters_features), dtype=int)

  #print("cluster features", clusters_features)
  #print("labels features", labels_features)

  # Compute pseudolabels only for new digits
  err = 0 # Initialize error counter
  for i in range(len(clusters_features) - batch_size, len(clusters_features)):
    pseudolabels_features[i] = map_clu2lbl[clusters_features[i]]

    # print(pseudolabels_features[i], labels_features[i], clusters_features[i])
    if pseudolabels_features[i] != labels_features[i]:
      err += 1
  
  # print(pseudolabels_features)
  # print(labels_features[len(clusters_features)-batch_size:len(clusters_features)])

  # conf_matrix(clusters_features_saved, labels_features_saved_init, , labels_init_list)
  
  # Evaluation metrics
  # ComputeEvalMetrics(labels_features, pseudolabels_features, cluster_list, labels_init_list)

  return pseudolabels_features, err

# New code - need fix

In [ ]:
# NEW CODE --- NEED TESTS
'''Function to compute kmean clustering on the new dataset and the saved features'''
def k_mean_clustering(features_run, labels_run, features_saved, labels_saved, n_cluster, batch_size):
  
  # Convert list to nparray
  #features = np.array(features_saved)
  # features = features.astype('float32')
  #labels = np.array(labels_saved)  

  labels_init_list = list(range(0, n_cluster)) # TEMPORARY

  # Concateno al vettore delle features iniziali le features della nuova batch da analizzare
  features = np.concatenate((features_saved, features_run))
  labels = np.append(labels_saved, labels_run).astype(int)

  # Repeat until clustering is correct
  max_iter = 20
  iter = 0
  while True:
    # KMean Clustering
    k_mean = create_k_mean(features, n_cluster)

    # Find pseudolabels for each new image
    # Pseudolabels are computed by looking at the confusion matrix of the saved dataset (where ground truth is known)
    clusters_saved = list(k_mean.labels_[0:len(labels_saved)])
    labels_saved = list(labels_saved)
    cluster_list = list(range(0,n_cluster))
    map_clu2lbl, map_lbl2clu = cluster_to_label(clusters_saved, labels_saved, cluster_list, labels_init_list) 
 
    if len(map_clu2lbl) == n_cluster or iter > max_iter:
        if(iter > max_iter):
          print("Clustering did not converge. Skipping batch.NOT WORKING ---- NEED FIX-----")
        # Exit the loop
        break

  clusters_features = k_mean.labels_

  # Find pseudo label (labels obtained from the model of each image
  pseudolabels = np.zeros(len(clusters_features), dtype=int)

  # Compute pseudolabels only for new digits
  err = 0 # Initialize error counter
  for i in range(len(clusters_features) - batch_size, len(clusters_features)):
    pseudolabels[i] = map_clu2lbl[clusters_features[i]]

    #print(pseudolabels[i], labels[i], clusters_features[i])
    if pseudolabels[i] != labels[i]:
      err += 1
      print("label:",labels[i] ,"pseudolabel:", pseudolabels[i])
  
  # Evaluation metrics
  # ComputeEvalMetrics(labels_features, pseudolabels_features, labels_init_list)


  return pseudolabels, err

''' Function to check if the current label is already known to the model. If not it augments the custom layer adding a new node'''
def CheckLabelKnown(model, current_label):
    
    found = False
    
    for i in range(0, len(model.label)):
        if(current_label == model.label[i]):
            found = True
          
    # If the label is not known
    if not found:
       #  print(f'\n\n    New digit detected -> digit \033[1m{current_label}\033[0m \n')
        print("new digit detected", current_label )

        model.label.append(current_label)   # Add new digit to label
                
        # Increase weights and biases dimensions
        model.W = np.hstack((model.W, np.zeros([model.W.shape[0],1])))
        model.b = np.hstack((model.b, np.zeros([1])))
        
        model.W_2 = np.hstack((model.W_2, np.zeros([model.W.shape[0],1])))
        model.b_2 = np.hstack((model.b_2, np.zeros([1])))


def update_active_layer(model, features, pseudo_label):

    learn_rate = model.l_rate

    CheckLabelKnown(model, pseudo_label)
    
    y_true_soft = DigitToSoftmax(pseudo_label, model.label)
               
    # Prediction
    y_pred = model.predict(features)
        
    # Backpropagation
    cost = y_pred-y_true_soft
        
    for j in range(0,model.W.shape[0]):

         # Update weights
        dW = np.multiply(cost, features[j]*learn_rate)
        model.W[j,:] = model.W[j,:]-dW

    # Update biases
    db      = np.multiply(cost, learn_rate)
    model.b = model.b-db
           
    # the next part is only to plot the confusion matrix
    # if the train data is finished still train the model but do not save the results
    #if(i>=train_samples):

    #    y_true_soft = DigitToSoftmax(y_tot[i], model.label)
                   
        # Find the max iter for both true label and prediction
    #    if(np.amax(y_true_soft) != 0):
    #        max_i_true = np.argmax(y_true_soft)

    #    if(np.amax(y_pred) != 0):
    #        max_i_pred = np.argmax(y_pred)






def trainOneEpoch_OL(model, images, labels, features_saved, labels_saved, batch_size):
    
    n_cluster = 10
    n_samples = images.shape[0]

    # Features extraction
    # features = model.ML_frozen.predict(images.reshape((1,28,28,1)), verbose = False)




    # Define initial set of features
    labels_init_list = list(range(0, n_cluster))

    # labels_init_list = list([1, 9, 5, 0])
    # labels_init_list = list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    # n_cluster = len(labels_init_list)

    # Extract from the saved features the labels that we need
    features_saved_sel = []
    labels_saved_sel = []
    # Extract features of digits considered in labels_init_list
    for i in range(0, len(features_saved)):
        if labels_features_saved[i] in labels_init_list:
            features_saved_sel.append(features_saved[i,:])
            labels_saved_sel.append(labels_saved[i])


    # Split dataset in batches
    n_batch = int(np.ceil(n_samples / batch_size))
    images_batch = np.array_split(images, n_batch)
    labels_batch = np.array_split(labels, n_batch)


    err_tot = 0
    pseudo_labels = np.empty
    for i in range(0, n_batch):
        print("Starting batch:" + str(i) + "/" + str(n_batch))
        # Features extraction
        start1 = time.time()
        features = model.ML_frozen.predict(images_batch[i].reshape((batch_size,28,28,1)), verbose = False) # VERIFICO
        end1 = time.time()

        # Kmean clustering
        start2 = time.time()
        pseudo_label, err = k_mean_clustering(features, labels_batch, features_saved_sel, labels_saved_sel, n_cluster, batch_size)
        end2 = time.time()
        pseudo_labels = np.append(pseudo_labels, pseudo_label.astype(int))
        err_tot += err

        print("Features extraction took {:f} seconds".format(end1 - start1), "and Kmean clustering took {:f} seconds".format(end2 - start2), ", with {}".format((1-err/features.shape[0])*100), "%", "accuracy", "({} errors)".format(err))

    print("The error in clustering is: {}".format(int(err_tot/n_samples*100)), '%')
    print("Errors:", err_tot, "Samples: ", n_samples)

    # ONLINE-LEARNING

    print('**********************************\n Performing training with OL\n')

    for i in range(0, n_samples):
        pass
        # update_active_layer(model, features[i,:], pseudo_labels[i])





digits = np.concatenate((digits_train, digits_test))   
labels = np.concatenate((label_digits_train, label_digits_test))  
trainOneEpoch_OL(Model_OL, digits, labels, features_saved, labels_features_saved, batch_size)
